## Shape 

In [1]:
import tvm 
from tvm import te
import numpy as np


n = te.var(name = 'n')
type(n)


[13:21:34] /home/aditya/Downloads/tvm/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.3 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
[13:21:34] /home/aditya/Downloads/tvm/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.3 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
[13:21:34] /home/aditya/Downloads/tvm/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.3 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`


tvm.tir.expr.Var

A New Shape can be a tuple (m,n) and can be acccessed using the indices i, j. these shapes can be computed using expressions and operators.

In [2]:
# Matrix Transpose

n = te.var('n')
m = te.var('m')

A = te.placeholder((n,m), name= "A")
B = te.compute((m,n), lambda i,j : A[j,i], name= "B")

te_schedule = te.create_schedule(B.op)

tvm.lower(te_schedule, [A,B], name = "Matrix Transpose")


# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def Matrix Transpose(A: T.handle, B: T.handle):
        T.func_attr({"from_legacy_te_schedule": T.bool(True), "tir.noalias": T.bool(True)})
        n, m = T.int32(), T.int32()
        A_1 = T.match_buffer(A, (n, m), strides=("stride", "stride"), buffer_type="auto")
        B_1 = T.match_buffer(B, (m, n), strides=("stride", "stride"), buffer_type="auto")
        for i, j in T.grid(m, n):
            B_2 = T.Buffer((B_1.strides[0] * m,), data=B_1.data, buffer_type="auto")
            A_2 = T.Buffer((A_1.strides[0] * n,), data=A_1.data, buffer_type="auto")
            B_2[i * B_1.strides[0] + j * B_1.strides[1]] = A_2[j * A_1.strides[0] + i * A_1.strides[1]]

In [5]:
a = np.arange(16,dtype='float32').reshape((4,4))
b = np.empty((4,4), dtype = 'float32')

a,b = tvm.nd.array(a), tvm.nd.array(b)

mod = tvm.build(te_schedule,[A,B])

mod(a,b)

print(a)
print(b)




[[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]
 [ 8.  9. 10. 11.]
 [12. 13. 14. 15.]]
[[ 0.  4.  8. 12.]
 [ 1.  5.  9. 13.]
 [ 2.  6. 10. 14.]
 [ 3.  7. 11. 15.]]
